In [68]:
import sounddevice as sd
from sidekit.frontend.features import plp, compute_delta, mfcc
import numpy as np
import librosa
from tensorflow.keras.models import load_model

In [127]:
## Loadding model
model = load_model('resnet_best.h5')
#model = load_model('best_of_the_best_model_lstm.h5')
# resnet dictionary
classes = {'kab': 0, 'fa': 1, 'en': 2, 'fr': 3, 'ru': 4, 'es': 5, 'it': 6, 'de': 7}
#lstm dictionary
#classes = {'persian': 0, 'english': 1, 'german': 2, 'spanish': 3, 'italian': 4, 'kabile': 5, 'russian': 6, 'french': 7}

In [173]:
#record sound
duration = 2.5  # seconds
fs = 16000
myrecording = sd.rec(int(duration * fs*1.5), samplerate=fs, channels=1)
sd.wait()
myrecording = myrecording.reshape(-1)
myrecording, _ = librosa.effects.trim(myrecording)
myrecording = myrecording[:int(duration*fs)]

In [174]:
# play sound
sd.play(myrecording, fs)

In [123]:
# feature extraction
# for LSTM
feat_item = plp(myrecording, fs=fs, rasta=False, nwin=.128, shift=.032)[0]
feat_delta1 = compute_delta(feat_item)
feat_delta2 = compute_delta(feat_delta1)
feat = np.concatenate((feat_item, feat_delta1, feat_delta2), axis=1).T
feat = ((feat - np.mean(feat))/np.var(feat)).reshape(*feat.shape, 1).T
print(feat.shape)

(1, 75, 39)


In [175]:
# feature extraction
# for CNN & ResNet
feat_item = librosa.feature.melspectrogram(myrecording, sr=fs)
feat = librosa.power_to_db(feat_item, ref=np.max).reshape((1,*feat_item.shape, 1))
feat = ((feat - np.mean(feat))/np.var(feat))
print(feat.shape)

(1, 128, 79, 1)


In [176]:
prediction = model.predict(feat)
print(list(classes.keys())[np.argmax(prediction[0])])

en


In [177]:
prediction

array([[3.6559118e-06, 2.5280264e-07, 9.8529577e-01, 1.3632525e-04,
        2.8389775e-09, 1.0657275e-04, 3.6096011e-05, 1.4421425e-02]],
      dtype=float32)